# Extracción, transformación y carga de los datos

### Librerias útiles

Una vez que se cuenta con el conjunto de datos etiquetado, éste debe ser dividido en tres subconjuntos que llamaremos "entrenamiento", "validación" y "prueba", divididos en 70%, 10% y 20% de los datos. Esta división es común dentro de la ciencia de datos y está basada en el *principio de Pareto*.

Además de la división es necesario transformar el texto de cada tweet, pues recordemos cada texto puede contener emoticonos 🇧🇷🇨🇷🇳🇮🇲🇽, hashtags *#VivaLaMigración*, menciones a usuarios [@el_BID](https://twitter.com/el_bid?lang=es) y links a páginas externas. Algunos de estos elementos pueden contener información útil para la detección de sentimiento y/o xenofobia (como los hashtags), pero otros pueden no ser nada informativos (como los nombres de usuario).

##### Scikit learn
Se trata de una librería de código abierto enfocada en proveer herramientas de aprendizaje de máquinas tales como modelos estadísticos y matemáticos, así como métricas de evaluación comunes en algoritmos de aprendizaje de máquinas.
<figure>
    <img src="./assets/images/scikit.png"
         alt="scikit-learn logo"
         style="max-width: 20%; height: auto">
    <figcaption>scikit-learn logo</figcaption>
</figure>

Esta librería nos permitirá dividir el conjunto de datos en los subconjuntos mencionados.

In [31]:
#Imports

#data reading
import pandas as pd

#data processing
from pysentimiento.preprocessing import preprocess_tweet

#math
import numpy as np

#scikit-learn metrics
from sklearn.metrics import (
    confusion_matrix, recall_score, accuracy_score, recall_score, precision_score, f1_score, classification_report
)

### Cargando los datos etiquetados

In [32]:
data = pd.read_csv('./assets/data/xeno_data_workshop.csv')
data.sample(n=5)

,id,text,date,label
8918,1222281676593778688,"@MigracionCol Buenas tardes, quisiera saber qu...",2021-11-01,0
3122,1178366723944263680,En lo tiempos de bonanza de mi #Venezuela le h...,2021-11-24,0
922,1289992793285373952,Existen dos tipos de migrantes venezolanos tóx...,2021-10-06,0
3423,1267622653168418816,@HaroldoHerrera El tipo dijo que ningún país v...,2021-09-06,0
3674,1174815406952984320,@ EJDS0210 oh hijo de puta inmigrante venezola...,2021-09-07,1


In [34]:
print('Tweet marcado como xenofóbo:\n {}\nTweet marcado como NO xenofóbo:\n {}'.format(data[data.label==1].sample(n=1).text.values[0], data[data.label==0].sample(n=1).text.values[0]))

Tweet marcado como xenofóbo:
 @djinthemix2 @ELTIEMPO @ClaudiaLopez Con esa logica de defender a los Venecos, también revoquemos al presidente, que quiere llenar a colombia de esa plaga, solo para poder sacar tajada de las ayudas que dan otros países
Tweet marcado como NO xenofóbo:
 Que opinas de la reformas migratorias?


### Limpieza de textos

In [66]:
example_text = 'Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja.'
bid_url = ' Página oficial del BID:          https://www.iadb.org/es'
example_text = example_text + bid_url
print(example_text)

Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?
Yo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja. Página oficial del BID:          https://www.iadb.org/es


In [67]:
preprocess_tweet(example_text, user_token='@usuario', url_token='url', preprocess_hashtags=True, demoji=True, shorten=2, normalize_laughter=True, hashtag_token='hashtag')

'Que opinas de la reformas Migratorias? emoji ojos emoji  emoji ojos emoji  mi buen amigo @usuario?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes  emoji persona de mostrador de información emoji  emoji bandera brasil emoji  emoji bandera costa rica emoji  emoji bandera nicaragua emoji  emoji bandera méxico emoji  emoji bíceps flexionado emoji hashtag yo te apoyo eexx jaja. Página oficial del BID:  url'

Podemos observar que el texto cambia los links por la palabra definida "url", los nombres de usuario por "@usuario", separa las palabras contenidas en un hashtag y además agrega "hashtag", normaliza las expresiones de risa y las letras repetidas. Describe el contenido de cada emoticono.

Las palabras definidas serán muy útiles para el modelo, pues éste puede ser configurado para que considere dichas palabras como elementos estructurales y no como palabras.

Sin embargo, se observa que el texto contiene espacios, mantiene los acentos, los saltos de línea y las mayúsculas. Estas características podrían ser no deseadas según el objetivo que se tiene y el modelo que se empleará para clasificar los textos. Ya que existen modelos que hacen diferencia entre estas características y otros que no.